# Update database with manually uploaded clips
Script to populate the koster lab database with information of clips that have been manually uploaded.

## Overview
We first download information from all subjects uploaded to the Koster lab project (#9747) using the [Python SDK for Panoptes!](https://github.com/zooniverse/panoptes-python-client). Then, we select those subjects manually uploaded and update the kosted db with the filename of the original movie, when the clip starts and the subject id.
Note, only Zooniverse project collaborators can retrieve subjects information from the Koster lab Zooniverse project.

### Import required packages

In [1]:
import os
import requests
#import numpy
import pandas as pd
import io
import json
from panoptes_client import Project, Panoptes
from datetime import datetime

### Specify project-specific info

In [2]:
# Connect to Zooniverse with your username and password
Panoptes.connect(username='', password='')

# Specify the project number of the koster lab
project = Project(9747)

# Specify the last and first dates when subjects were manually uploaded
last_date = '2020-02-03 20:30:00 UTC'
first_date = '2019-11-17 00:00:00 UTC'


### Download subject information from Zooniverse

In [3]:
# Get the export subjects
export = project.get_export('subjects')

# Save the response as pandas data frame
rawdata = pd.read_csv(
    io.StringIO(export.content.decode("utf-8")),
    usecols=[
        "subject_id",
        "metadata",
        "created_at",
        "workflow_id",
        "subject_set_id",
        "classifications_count",
        "retired_at",
        "retirement_reason",
    ],
)

### Select manually uploaded clip subjects 

In [4]:
# Filter manually uploaded subjects
man_data = rawdata[
    (last_date >= rawdata.created_at) & (first_date <= rawdata.created_at)
]

# filter clip subjects
man_data = man_data[man_data["metadata"].str.contains(".mp4")].reset_index()

### Flatten the subjects metadata information

In [5]:
# flatten the metadata information
flat_metadata = pd.json_normalize(man_data.metadata.apply(json.loads))

# Select the filename of the clips
clip_filenames = flat_metadata["filename"]

# Get the starting time of clips in relation to the original movie
# split the filename, select the last section, and remove the extension type
flat_metadata["start_time"] = (
    clip_filenames.str.rsplit("_", 1).str[-1].str.replace(".mp4", "")
)

# Extract the filename of the original movie
flat_metadata["movie_filename"] = flat_metadata.apply(
    lambda x: x["filename"].replace("_" + x["start_time"], ""), axis=1
)

# Get the end time of clips in relation to the original movie
flat_metadata["start_time"] = pd.to_numeric(
    flat_metadata["start_time"], downcast="signed"
)
flat_metadata["end_time"] = flat_metadata["start_time"] + 10

# select only relevant columns
flat_metadata = flat_metadata[
    ["filename", "movie_filename", "start_time", "end_time"]
]

### Include movie_ids 
Retrieve "id" and "filename" from the "movies" table of the koster db to add movie "flat_metadata"

In [ ]:
# Retrieve the id and filename from the movies table
#flat_metadata["movie_id"] = flat_metadata.apply(lambda x: get_id(x), 1)

### Compile all the information and update the clips and subjects tables of koster db

In [6]:
# Drop metadata column and define clip creation date as time uploaded to Zooniverse
man_data = man_data.drop(columns="metadata")

# Combine the information
comb_data = pd.concat([man_data, flat_metadata], axis=1)

# Select information to include in the clips table
clips = comb_data.drop(
    columns=[
        "subject_id",
        "movie_filename",
        "workflow_id",
        "subject_set_id",
        "classifications_count",
        "retired_at",
        "retirement_reason",
    ]
).rename(columns={"created_at": "clipped_date", "index": "id"})


# Combine the info to include in the subjects table
subjects = comb_data.rename(
    columns={
        "created_at": "zoo_upload_date",
        "index": "clip_id",
        "retirement_reason": "retirement_criteria",
        "subject_id": "id",
    }
)

subjects = subjects[
    [
        "id",
        "workflow_id",
        "subject_set_id",
        "classifications_count",
        "retired_at",
        "retirement_criteria",
        "zoo_upload_date",
        "clip_id",
    ]
]


#update the tables
